In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import json
from huggingface_hub import login

from tqdm import tqdm
import torch
from transformers import AutoTokenizer, pipeline

import pandas as pd
import numpy as np
import re

from baseline import *

# Read the JSON config file
with open('config.json', 'r') as f:
    config = json.load(f)

# Get the token from the JSON file
hg_token = config['HuggingFace']['token']
# Login using the token
login(token=hg_token)

# LLM folder
llm_folder = "/PHShome/jn180/llm_public_host"
# Data folder
data_folder = "/PHShome/cs1839/capstone_data/"
# results table path
results_df_path = data_folder + "results.csv"

# data to inference 
medication_status_test = pd.read_csv(data_folder + "medication_status_test.csv")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /PHShome/cs1839/.cache/huggingface/token
Login successful


# Pipeline

In [30]:
name_model_paths ={   
    # "Bio_ClinicalBERT": "/PHShome/jn180/llm_public_host/Bio_ClinicalBERT",

    # "Llama-3.1-8B": "/netapp3/raw_data3/share/llm_public_host/Llama-3.1-8B",
    "Llama-3.1-8B-Instruct": "/netapp3/raw_data3/share/llm_public_host/Llama-3.1-8B-Instruct",

    "Llama-3.2-1B-Instruct": "/netapp3/raw_data3/share/llm_public_host/Llama-3.2-1B-Instruct",
    "Llama-3.2-3B-Instruct": "/netapp3/raw_data3/share/llm_public_host/Llama-3.2-3B-Instruct",

    "Qwen2-7B-Instruct": "/PHShome/jn180/llm_public_host/Qwen2-7B-Instruct",
    "Qwen2.5-14B-Instruct": "/netapp3/raw_data3/share/llm_public_host/Qwen2.5-14B-Instruct",

    "meditron-7b": "/PHShome/jn180/llm_public_host/meditron-7b",

    # "Mistral-7B-Instruct-v0.3": "/netapp3/raw_data3/share/llm_public_host/Mistral-7B-Instruct-v0.3"

}

import os
# Set the environment variable to specify the GPUs

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


name_dataset = "MIT"
data_folder = "/PHShome/cs1839/capstone_data/"
results_df_path = data_folder + "results.csv"
medication_status_test = pd.read_csv(data_folder + "medication_status_test.csv")

# prompt_template = """
# Identify and categorize the medications mentioned in the following medical note. Extract all medications the patient has taken before, is currently taking, and any other medications mentioned.
# Note: Adjust the number of medications in each category based on the input. Write None if no other medication mentioned. Strictly follow the output format.
# Expected Output Format:
# "
# - Current Medications (Active): Medication_1, Medication_2
# - Discontinued Medications: Medication_3, Medication_4
# - Other Mentioned Medications (neither active nor discontinued): Medication_5, Medication_6
# END"

# Input Medical Note:
# {}

# Output:
# """

prompt_template = """
Input Medical Note:
{}

Create a bulleted list of which medications are mentioned and whether they are active, discontinued, or neither.

Expected Output Format:
"
- Current Medications (Active): Medication_1, Medication_2
- Discontinued Medications: Medication_3, Medication_4
- Other Mentioned Medications (neither active nor discontinued): Medication_5, Medication_6
END"

Output:
"""

for model_name, model_path in name_model_paths.items():
    df = run_pipeline(model_path=model_path,
                        input_df=medication_status_test[medication_status_test['index']==96],
                        prompt_template=prompt_template,
                        batch_size=16,
                        max_token_output=80,
                        use_sampling=False)
df

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


,index,snippet,active_medications,discontinued_medications,neither_medications,active_medications_pred,discontinued_medications_pred,neither_medications_pred,extraction_precision,extraction_recall,conditional_accuracy,conditional_macro_f1,conditional_macro_precision,conditional_macro_recall
0,96,She did have a capsule study done during her p...,"[imuran, remicade]",[6 mp],[],[iv imuran],[6 mp],"[remicade, capsule study]",0.5,0.666667,0.5,0.333333,0.333333,0.333333


# Result

# Metrics

## Task 1: Medication Extraction

- **Precision**: Measures the proportion of correctly predicted medications out of all predicted medications.

$$
  \text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
  $$

- **Recall**: Measures the proportion of correctly predicted medications out of all actual medications.

 $$
  \text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
  $$

## Task 2: Status Classification

- **Conditional Accuracy**: Measures the proportion of correct status predictions out of all correctly extracted medications from Task 1.
  $$
  \text{Conditional Accuracy} = \frac{\text{Correct Predictions for the Classes}}{\text{Total Correctly Extracted Medications from Task 1}}
  $$

- **Conditional Macro F1**: Combines precision and recall for each status class, calculates the F1-score for each, then averages them across classes.
  $$
  \text{F1\text{-}score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
  $$

---

# Example
## Task 1

| Active Medication | Discontinued Medication | Active Medication (Predicted) | Discontinued Medication (Predicted) |
|-------------------|-------------------------|-------------------------------|-------------------------------------|
| A                 | B                       | A                             | C                                   |


True Set: A, B

Pred Set: A, C



Precision = 1/2

Recall = 1/2


## Task 2
conditional metrics will only consider: A (C is not correctly extracted, removed)
| Active Medication | Discontinued Medication | Active Medication (Predicted) | Discontinued Medication (Predicted) |
|-------------------|-------------------------|-------------------------------|-------------------------------------|
| A                 | B                       | A                             |                                     |

conditional_accuracy = 1/2 

conditional_precision:
- Active: 1
- **Discountinued: 1**

conditional_recall:
- Active: 1
- Discountinued: 0



| Active Medication | Discontinued Medication | Active Medication (Predicted) | Discontinued Medication (Predicted) |
|-------------------|-------------------------|-------------------------------|-------------------------------------|
| A, C              |                         | A                             |C                                    |
| A                 |B, C                     | A                             |C                                    |
| A, B              |                         |                               |                                     |

conditional_acc =（A+A+C）/ (A+C+A+C) = 3/4 

conditional_precision_active = (A+A)/(A+A) = 1

conditional_precision_discountinued = C / (C+C) = 1/2

conditional_recall_active = (A+A)/ (A+A+A+B+c) = 2/5

conditional_recall_discountinued = (C)/ (B+C) = 1/2


In [17]:
import pandas as pd
from baseline import calculate_metrics_by_dataset

# Example usage
data = {
    'active_medications': [['A', 'C'], ['A'], ['A', 'B']],
    'discontinued_medications': [['E'], ['B', 'C'], []],
    'neither_medications': [['D'], [], []],
    'active_medications_pred': [['A','E'], ['A'], ['A','B']],
    'discontinued_medications_pred': [['C','D'], ['C'], []],
    'neither_medications_pred': [[], [], ['E']]
}

# Create the DataFrame
mimic_iv = pd.DataFrame(data)

# Run the function on the dataset
extraction_precision, extraction_recall, extraction_f1, conditional_accuracy, conditional_macro_f1, conditional_macro_precision, conditional_macro_recall = calculate_metrics_by_dataset(mimic_iv, 'MIMIC')

# Print the results
print(f"Extraction Precision: {extraction_precision:.3f}")
print(f"Extraction Recall: {extraction_recall:.3f}")
print(f"Extraction F1: {extraction_f1:.3f}")
print(f"Conditional Accuracy: {conditional_accuracy:.3f}")
print(f"Conditional Macro Precision: {conditional_macro_precision:.3f}")
print(f"Conditional Macro Recall: {conditional_macro_recall:.3f}")
print(f"Conditional Macro F1: {conditional_macro_f1:.3f}")

mimic_iv[['active_medications', 'discontinued_medications', 'neither_medications', 'active_medications_pred', 'discontinued_medications_pred', 'neither_medications_pred']]

Extraction Precision: 0.889
Extraction Recall: 0.889
Extraction F1: 0.889
Conditional Accuracy: 0.556
Conditional Macro Precision: 0.378
Conditional Macro Recall: 0.378
Conditional Macro F1: 0.378


,active_medications,discontinued_medications,neither_medications,active_medications_pred,discontinued_medications_pred,neither_medications_pred
0,"[A, C]",[E],[D],"[A, E]","[C, D]",[]
1,[A],"[B, C]",[],[A],[C],[]
2,"[A, B]",[],[],"[A, B]",[],[E]


In [30]:
# Load the data and sort it by the specified columns
import pandas as pd

# Data folder
data_folder = "/PHShome/cs1839/capstone_data/"
# results table path
results_df_path = data_folder + "results.csv"

result_df = pd.read_csv(results_df_path).round(3)

result_df = result_df._append({'Prompt': 'Create a bulleted list of which medications are mentioned and whether they are active, discontinued, or neither.',
                   'Dataset': 'MIT', 
                   'Model': 'GPT-3 + R(32 LOC)(0-Shot)',
                   'extraction_precision': 0.87,
                   'extraction_recall': 0.83,
                   'extraction_f1': round(2 * 0.87 * 0.83 / (0.87 + 0.83),3),
                   'conditional_accuracy': 0.85,
                   'conditional_macro_f1': 0.69,
                   'conditional_macro_precision': '--',
                   'conditional_macro_recall': '--'}, ignore_index=True)


result_df = result_df._append({'Prompt': 'Create a bulleted list of which medications are mentioned and whether they are active, discontinued, or neither.',
                   'Dataset': 'MIT', 
                   'Model': 'GPT-3 + R(8 LOC)(1-Shot)',
                   'extraction_precision': 0.90,
                   'extraction_recall': 0.92,
                    'extraction_f1': round(2 * 0.90 * 0.92 / (0.90 + 0.92),3),
                   'conditional_accuracy': 0.89,
                   'conditional_macro_f1': 0.62,
                   'conditional_macro_precision': '--',
                   'conditional_macro_recall': '--'}, ignore_index=True)

result_df[result_df.Dataset != 'Internal Data'].\
sort_values(
    by=['Prompt', 'Dataset', 'extraction_f1', 'conditional_accuracy', 'conditional_macro_f1'],
    ascending=[True, False, False, False, False] 
).set_index(['Prompt', 'Dataset', 'Model'])


extraction_precision  \
Prompt                                             Dataset  Model                                              
Create a bulleted list of which medications are... MIT      GPT-3 + R(8 LOC)(1-Shot)                   0.900   
                                                            GPT-3 + R(32 LOC)(0-Shot)                  0.870   
Input Medical Note:\n{}\n\nCreate a bulleted li... MIT      Llama-3.1-70B-Instruct                     0.788   
                                                            Llama-3.1-8B-Instruct                      0.784   
                                                            Llama-3.1-8B                               0.829   
                                                            Qwen2.5-32B-Instruct                       0.773   
                                                            meditron-70b                               0.727   
                                                            Mistral-Nemo-Instruct-2407                 0.753   
                                                            Qwen2.5-14B-Instruct                       0.731   
                                                            Qwen2-7B-Instruct                          0.689   
                                                            Qwen2.5-72B-Instruct                       0.712   
                                                            Mistral-7B-Instruct-v0.3                   0.613   
                                                            Qwen2-72B-Instruct                         0.553   
                                                            Llama-3.2-3B-Instruct                      0.454   
                                                            Llama-3.2-1B-Instruct                      0.395   
                                                            meditron-7b                                0.015   
                                                   MIMIC-IV Llama-3.1-8B                               0.710   
                                                            Llama-3.1-8B-Instruct                      0.612   
                                                            Mistral-Nemo-Instruct-2407                 0.614   
                                                            Llama-3.1-70B-Instruct                     0.601   
                                                            Qwen2.5-14B-Instruct                       0.586   
                                                            Qwen2.5-32B-Instruct                       0.600   
                                                            Qwen2.5-72B-Instruct                       0.604   
                                                            Qwen2-7B-Instruct                          0.526   
                                                            meditron-70b                               0.536   
                                                            Llama-3.2-3B-Instruct                      0.401   
                                                            Qwen2-72B-Instruct                         0.469   
                                                            Mistral-7B-Instruct-v0.3                   0.409   
                                                            Llama-3.2-1B-Instruct                      0.310   
                                                            meditron-7b                                0.032   

                                                                                        extraction_recall  \
Prompt                                             Dataset  Model                                           
Create a bulleted list of which medications are... MIT      GPT-3 + R(8 LOC)(1-Shot)                0.920   
                                                            GPT-3 + R(32 LOC)(0-Shot)               0.830   
Input Medical Note:\n{}\n\nCreate a bulleted li... MIT      Llama-3

In [32]:
result_df[result_df.Dataset == 'Internal Data'].\
sort_values(
    by=['Prompt', 'Dataset', 'extraction_recall', 'conditional_accuracy', 'conditional_macro_f1'],
    ascending=[True, False, False, False, False] 
).set_index(['Prompt', 'Dataset', 'Model'])

extraction_precision  \
Prompt                                             Dataset       Model                                              
Input Medical Note:\n{}\n\nCreate a bulleted li... Internal Data Qwen2.5-14B-Instruct                       0.209   
                                                                 meditron-70b                               0.320   
                                                                 Mistral-Nemo-Instruct-2407                 0.498   
                                                                 Llama-3.1-70B-Instruct                     0.579   
                                                                 Llama-3.1-8B                               0.183   
                                                                 Llama-3.2-3B-Instruct                      0.186   
                                                                 Qwen2-7B-Instruct                          0.227   
                                                                 Qwen2.5-32B-Instruct                       0.344   
                                                                 Qwen2.5-72B-Instruct                       0.771   
                                                                 Mistral-7B-Instruct-v0.3                   0.195   
                                                                 Llama-3.1-8B-Instruct                      0.302   
                                                                 meditron-7b                                0.083   
                                                                 Qwen2-72B-Instruct                         0.229   
                                                                 Llama-3.2-1B-Instruct                      0.095   

                                                                                             extraction_recall  \
Prompt                                             Dataset       Model                                           
Input Medical Note:\n{}\n\nCreate a bulleted li... Internal Data Qwen2.5-14B-Instruct                    0.840   
                                                                 meditron-70b                            0.761   
                                                                 Mistral-Nemo-Instruct-2407              0.708   
                                                                 Llama-3.1-70B-Instruct                  0.680   
                                                                 Llama-3.1-8B                            0.667   
                                                                 Llama-3.2-3B-Instruct                   0.638   
                                                                 Qwen2-7B-Instruct                       0.588   
                                                                 Qwen2.5-32B-Instruct                    0.565   
                                                                 Qwen2.5-72B-Instruct                    0.546   
                                                                 Mistral-7B-Instruct-v0.3                0.461   
                                                                 Llama-3.1-8B-Instruct                   0.435   
                                                                 meditron-7b                             0.203   
                                                                 Qwen2-72B-Instruct                      0.202   
                                                                 Llama-3.2-1B-Instruct                   0.077   

                                                                                             extraction_f1  \
Prompt                                             Dataset       Model                                       
Input Medical Note:\n{}\n\nCreate a bulleted li... Internal Data Qwen2.5-14B-Instruct                0.335   
                                                                 